<a href="https://colab.research.google.com/github/pragnavi/Machine-Learning/blob/master/pr2370_Q4_Insult_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insult Classification

In this exercise, we would like to filter out insulting comments on a web forum.

To train our models, we have a list of historic comments with a judgement wether they're insulting or not.

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, roc_auc_score

In [ ]:
drive.mount('/content/drive')
path_to_insults = '/content/drive/MyDrive/Colab Notebooks/data/'
data = pd.read_csv(path_to_insults + 'train-utf8.csv')
data = data.dropna()
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Insult,Date,Comment
0,1,20120618192155Z,You fuck your dad.
1,0,20120528192215Z,i really don't understand your point. It seem...
4,0,20120619094753Z,Các bạn xuống đường biểu tình 2011 có ôn hoà k...
5,0,20120620171226Z,"@SDL OK, but I would hope they'd sign him to a..."
6,0,20120503012628Z,Yeah and where are you now?


In [ ]:
print ("%d comments, of which %d insults (%d%%)" % \
    (len(data), data.Insult.sum(), 100 * data.Insult.mean()))

3229 comments, of which 964 insults (29%)


### Looking for known bad words (10 pts)

One way to do this, is to load Google's bad word list and flag comments that contain one or more words.

- Load `google_badlist.txt` from `data/insults/`
- Add a column to `data` with a flag (0 or 1) if the comment contains a bad word
- Compute the accuracy of this method - does this look good?
- What would a naive classifier's score be (i.e., always predicting 0 or 1)?
- Also compute the precision, recall, F1 score and AUC score
- What is your verdict?

In [ ]:
filename = path_to_insults + 'google_badlist.txt'
filename
bad_words = np.loadtxt(filename, delimiter='\n', dtype = 'str')

In [ ]:
# Your code here
data['Bad_Words'] = data['Comment'].apply(lambda x: int(any(i in x.split() for i in bad_words)))
data.head(10)

,Insult,Date,Comment,Bad_Words
0,1,20120618192155Z,You fuck your dad.,1
1,0,20120528192215Z,i really don't understand your point. It seem...,0
4,0,20120619094753Z,Các bạn xuống đường biểu tình 2011 có ôn hoà k...,0
5,0,20120620171226Z,"@SDL OK, but I would hope they'd sign him to a...",0
6,0,20120503012628Z,Yeah and where are you now?,0
8,1,20120502173553Z,Either you are fake or extremely stupid...mayb...,0
9,1,20120620160512Z,That you are an idiot who understands neither ...,0
10,0,20120620015140Z,@jdstorm dont wish him injury but it happened ...,0
11,0,20120530044519Z,"Be careful,Jimbo.OG has a fork with your name ...",0
15,1,20120611090207Z,FOR SOME REASON U SOUND RETARDED. LOL. DAMN. W...,0


In [ ]:
print ("%d comments, of which %d contain bad words (%d%%)" % \
    (len(data), data.Bad_Words.sum(), 100 * data.Bad_Words.mean()))

3229 comments, of which 558 contain bad words (17%)


In [ ]:
x = np.array(data['Bad_Words'])
y = np.array(data['Insult'])

In [ ]:
aa = accuracy_score(x,y).mean()
print ("Accuracy: %d%%" % \
    (100 * aa))
cc = (data.Insult == 0).mean()
print ("Naive Classifier Score: ", cc)
pp = precision_score(x,y)
print("Precision: ", pp)
rr = recall_score(x,y)
print("Recall: ", rr)
ff = f1_score(x,y)
print("F1 Score: ", ff)
auc = roc_auc_score(x,y)
print("AUC Score: ", auc)

Accuracy: 67%
Naive Classifier Score:  0.7014555589965934
Precision:  0.24688796680497926
Recall:  0.4265232974910394
F1 Score:  0.3127463863337714
AUC Score:  0.5773574929986085


**Verdict:**<br>
Accuracy of this method is not promising. In this case, precision and recall are important as well to avoid false positives and false negatives. Obtained values of precision and recall are not good either. If we use the comments column as input other than extracting bad words the results might look better or worse. I've implementd the same below.

### Learning bad words on the fly (10 pts)

Another way of doing this, is to learn the insulting words on the fly using `CountVectorizer`.

Please refer to the scikit learn tutorial at 'http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html' if you need some help.

Here is what you need to do:

- Import `CountVectorizer` from `sklearn.feature_extraction.text`
- Train the `CountVectorizer` on the insults and create a feature set $X$ representing words in the comments
- Train `MultinomialNB` and `BernoulliNB` from `scikitsklearn`  on the new feature set $X$
- Using cross-validation, compute the accuracy, precision, recall, F1 and AUC of your model
- What is your verdict?

NOTE: The F1 score is another useful score to compute when one of the two classes is very rare. We didn't go over it in class but it's basically the harmonic mean between precision and recall and goes from 0 (min) to 1 (max).  You can see more here: 'https://en.wikipedia.org/wiki/F1_score'

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# Your code here
cv = CountVectorizer()

In [ ]:
X = cv.fit_transform(data['Comment'])
y = np.array(data['Insult'])

In [ ]:
model = BernoulliNB()
scores = ['accuracy','precision','recall','f1','roc_auc']
print('model: {}'.format(model))
model.fit(X,y)
for score in scores:
    mean_score = cross_val_score(model, X, y, scoring=score).mean()
    if(score == 'accuracy'):
      print('{}: {}%'.format(score, round(mean_score*100)))
    else:
      print('{}: {}'.format(score, mean_score))

model: BernoulliNB()
accuracy: 72%
precision: 0.5924841437632136
recall: 0.2137251727115717
f1: 0.31317717570962067
roc_auc: 0.7953246408037761


In [ ]:
model = MultinomialNB()
scores = ['accuracy','precision','recall','f1','roc_auc']
print('model: {}'.format(model))
model.fit(X,y)
for score in scores:
    mean_score = cross_val_score(model, X, y, scoring=score).mean()
    if(score == 'accuracy'):
      print('{}: {}%'.format(score, round(mean_score*100)))
    else:
      print('{}: {}'.format(score, mean_score))

model: MultinomialNB()
accuracy: 76%
precision: 0.5877351157530721
recall: 0.6815468480138169
f1: 0.6309042135477986
roc_auc: 0.80386857955217


**Verdict:**<br>
Multinomial classifier did better than Bernoulli in terms of accuracy. But for insult classification dataset, we also have to consider the precision and recall score. Although catching maximum number of insults is the goal of this prediction, passing a clear string as an insult is not acceptable as well. So, considering F1 scores alongside accuracy, Multinomial classifier performs better than Bernoulli classifier.

**Conclusion**<br>
Considering the scores obtained on both the methods, "Learning bad words on the fly" gives better results compared to "Looking for known bad words".